In [1]:
import tensorflow as tf
from numpy.random import RandomState

In [2]:
# 定义训练数据batch的大小
BATCH_SIZE=8

In [3]:
# 定义神经网络的参数
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

In [4]:
# 在shape的一个维度上使用None可以方便使用不大的batch大小。在训练时需要把数据分成比较小的batch，但是在测试时，可以一次性使用全部的数据。当数据集比较abs小时，这样比较方便测试，但是数据集比较大时，将大量数据放入一个batch可能会导致内存溢出
x=tf.placeholder(tf.float32,shape=(None,2),name='x-input')
y_=tf.placeholder(tf.float32,shape=(None,1),name='y-input')

In [5]:
# 定义神经网络前向传播过程
a=tf.matmul(x,w1)
y=tf.matmul(a,w2)

# 在上面的代码中，y_代表正确结果，y代表预测结果

In [6]:
# 定义损失函数和反向传播函数
# 定义损失函数来刻画预测值与真实值之间的差距
cross_entropy=-tf.reduce_mean(y_*tf.log(tf.clip_by_value(y,1e-10,1.0)))


In [7]:
# 定义学习率
learning_rate=0.001

In [9]:
# 定义反向传播算法来优化神经网络中的参数
train_step=tf.train.AdadeltaOptimizer(learning_rate).minimize(cross_entropy)

In [10]:
# 通过随机数生成一个模拟数据集
rdm=RandomState(1)
dataset_size=128
X=rdm.rand(dataset_size,2)

In [11]:
# 定义规则来给出样本的标签。这里所有x1+x2<1的样本都被认为是正样本，其他为负样本。这里使用0和1的表示方法
Y=[[int(x1+x2)<1] for (x1,x2) in X]

In [14]:
# 创建一个会话来运行tensorflow程序
with tf.Session() as sess:
    init_op=tf.global_variables_initializer()
    # 初始化变量
    sess.run(init_op)
    print('w1',sess.run(w1))
    print('w2',sess.run(w2))
    print('-'*90)
    # 设定训练轮数
    STEPS=5000
    for i in range(STEPS):
        # 每次选取batch_size个样本进行训练
        start=(i*BATCH_SIZE)%dataset_size
        end=min(start+BATCH_SIZE,dataset_size)
        
        # 通过选取的样本训练神经网络并更新参数
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        if i%1000==0:
            # 每个一段时间计算在所有数据上的交叉熵并输出
            total_cross_entropy=sess.run(cross_entropy,feed_dict={x:X,y_:Y})
            print('After %d training step(s),cross entropy on all data is %g'%(i,total_cross_entropy))
    print('-'*90)
    print('w1',sess.run(w1))
    print('w2',sess.run(w2))

w1 [[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
w2 [[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
------------------------------------------------------------------------------------------
After 0 training step(s),cross entropy on all data is 0.0677411
After 1000 training step(s),cross entropy on all data is 0.0676783
After 2000 training step(s),cross entropy on all data is 0.0676121
After 3000 training step(s),cross entropy on all data is 0.067546
After 4000 training step(s),cross entropy on all data is 0.0674799
------------------------------------------------------------------------------------------
w1 [[-0.81270415  1.485931    0.06668379]
 [-2.443958    0.10045876  0.5924246 ]]
w2 [[-0.8126457 ]
 [ 1.4860655 ]
 [ 0.06659038]]
